In [18]:
import pandas as pd
from postal.expand import expand_address
import numpy as np 
from helpers import s3_connection
import helpers 
import matplotlib.pyplot as plt

IMPORT DES DONNEES

In [33]:
df = pd.read_csv(
    "/Users/noelinecasteil/Documents/statapp/DPE/DPE_ADEME/dpe-v2-logements-existants.csv",
    sep=",",  # Séparateur CSV
    encoding="utf-8",
    low_memory=False)

df['Date_réception_DPE'].count()

4537525

In [3]:
s3 = s3_connection()

NameError: name 's3_connection' is not defined

In [4]:
path_logements_existants = "clichere/diffusion/DPE/DPE_ADEME/dpe-v2-logements-existants.csv"
df = s3.read_file_from_s3(path_logements_existants)

/home/onyxia/STATAPP/helpers.py:35: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_in, usecols=columns_to_select, dtype=dtype_spec, sep=sep)


In [54]:
df2 = pd.read_csv(
    "/Users/noelinecasteil/Documents/statapp/DPE/DPE_ADEME/dpe-v2-logements-neufs.csv",
    sep=",",  # Séparateur CSV
    encoding="utf-8",
    low_memory=False)

df2['Date_réception_DPE'].count()

537952

In [5]:
path_logements_existants = "clichere/diffusion/DPE/DPE_ADEME/dpe-v2-logements-existants.csv"
df2 = s3.read_file_from_s3(path_logements_existants)

/home/onyxia/STATAPP/helpers.py:35: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_in, usecols=columns_to_select, dtype=dtype_spec, sep=sep)


In [20]:
vf = pd.read_csv(
    "/Users/noelinecasteil/Documents/statapp/ValeursFoncieres/valeursfoncieres-2022.txt",
    sep="|",  
    encoding="utf-8",
    low_memory=False
)

In [6]:
path_valeursfoncieres_2022 = "clichere/diffusion/Valeursfoncières/valeursfoncieres-2022.txt"
vf = s3.read_file_from_s3(path_valeursfoncieres_2022, sep="|")

/home/onyxia/STATAPP/helpers.py:35: DtypeWarning: Columns (18,23,24,26,28,29,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_in, usecols=columns_to_select, dtype=dtype_spec, sep=sep)


In [21]:
# Convertir explicitement toutes les colonnes en chaînes
vf['Adresse'] = vf['No voie'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf['Type de voie'].fillna('').astype(str) + " " + \
                vf['Voie'].fillna('').astype(str) + ", " + \
                vf['Code postal'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf['Commune'].fillna('').astype(str)

vf['Adresse'] = vf['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides

In [22]:
def normalize_address(address):
    if pd.isna(address) or address.strip() == '':
        return None  
    try:
        normalized = expand_address(address)  
        return normalized[0] if normalized else None  # Ne garde que la première version
    except Exception as e:
        print(f"Erreur avec l'adresse '{address}': {e}")
        return None

In [23]:
df['Date_réception_DPE'] = pd.to_datetime(df['Date_réception_DPE'], errors='coerce')
df = df[df['Date_réception_DPE'].dt.year == 2022].copy()

In [24]:
vf = vf[vf['Code postal'].notna()]
df = df[df['N°_département_(BAN)'].notna()]

In [25]:
vf['Code postal'] = vf['Code postal'].astype(int).astype(str)
df.loc[:, 'N°_département_(BAN)'] = df['N°_département_(BAN)'].astype(str)

In [26]:
import re

def normalize_vf_address(address, code_postal):
    """ Normalise l'adresse et supprime l'arrondissement après 'PARIS' uniquement si le département est 75. """
    normalized_address = normalize_address(address)  

    if pd.notna(normalized_address) and str(code_postal).startswith("75"):
        # Supprime le numéro après "PARIS" uniquement pour le département 75
        normalized_address = re.sub(r'(paris) \d{2}$', r'\1', normalized_address, flags=re.IGNORECASE)
    
    return normalized_address


## TEST AVEC 5% POUR LES SURFACES

In [87]:
def test_match2(vf, df):
    merged = []


    #boucle par département
    for department in df['N°_département_(BAN)'].unique():  
        print(f"Traitement du département : {department}")
        
        #filtre
        vf_dept = vf[vf['Code postal'].astype(str).str.startswith(department)].copy()
        df_dept = df[df['N°_département_(BAN)']==department].copy()

        #normalisation adresses
        vf_dept['Adresse'] = vf_dept['Adresse'].str.strip().str.replace(r'\s+', ' ', regex=True)
        vf_dept['Adresse_Normalisee'] = vf_dept.apply(lambda row: normalize_vf_address(row['Adresse'], row['Code postal']), axis=1)
        df_dept['Adresse_Normalisee'] = df_dept['Adresse_(BAN)'].apply(normalize_address)

        adresse_counts = vf_dept['Adresse_Normalisee'].dropna().value_counts()
        unique1 = list(adresse_counts[adresse_counts == 1].index)
        doublons1 = list(adresse_counts[adresse_counts > 1].index)
        final = unique1 + doublons1
        set_final = set(final)

        adresse_counts2 = df_dept['Adresse_Normalisee'].dropna().value_counts()
        unique2 = list(adresse_counts2[adresse_counts2 == 1].index)
        doublons2 = list(adresse_counts2[adresse_counts2 > 1].index)
        final2 = unique2 + doublons2
        set_final2 = set(final2)

        commun = set_final.intersection(set_final2)

        vf_dept['Surface Carrez du 1er lot'] = pd.to_numeric(
            vf_dept['Surface Carrez du 1er lot'].astype(str).str.replace(',', '.'), errors='coerce'
        )
        df_dept['Surface_habitable_logement'] = pd.to_numeric(
            df_dept['Surface_habitable_logement'].astype(str).str.replace(',', '.'), errors='coerce'
        )

        for adresse in commun:
            dfsub = df_dept[df_dept['Adresse_Normalisee'] == adresse]
            vfsub = vf_dept[vf_dept['Adresse_Normalisee'] == adresse]

            for _, row2 in dfsub.iterrows():
                best_match = None
                best_value = -1

                for _, row1 in vfsub.iterrows():
                    surface1 = row1['Surface Carrez du 1er lot']
                    surface2 = row2['Surface_habitable_logement']

                    #si surfaces identiques on match direct
                    if surface1 == surface2:
                        best_match = row1
                        break

                    #ecart inf à 5%
                    if abs(surface1 - surface2) / max(surface1, surface2) < 0.05:
                        valeur_fonciere = pd.to_numeric(str(row1.get('Valeur fonciere', 0)).replace(',', '.'), errors='coerce')
                        if valeur_fonciere > best_value:
                            best_value = valeur_fonciere
                            best_match = row1

                if best_match is not None:
                    merged.append({**row2.to_dict(), **best_match.to_dict()})

    #df des résultats fusionnés
    df = pd.DataFrame(merged)
    return df

#application fonction
test2 = test_match2(vf, df)

Traitement du département : 13
Traitement du département : 92
Traitement du département : 69
Traitement du département : 03
Traitement du département : 56
Traitement du département : 75
Traitement du département : 29
Traitement du département : 91
Traitement du département : 62
Traitement du département : 21
Traitement du département : 11
Traitement du département : 02
Traitement du département : 59
Traitement du département : 94
Traitement du département : 44
Traitement du département : 50
Traitement du département : 37
Traitement du département : 81
Traitement du département : 67
Traitement du département : 93
Traitement du département : 83
Traitement du département : 63
Traitement du département : 95
Traitement du département : 76
Traitement du département : 33
Traitement du département : 78
Traitement du département : 38
Traitement du département : 51
Traitement du département : 57
Traitement du département : 77
Traitement du département : 35
Traitement du département : 54
Traiteme

In [88]:
test2.to_csv("testfinal.csv", index=False)

In [14]:
print(len(vf))

3764712


In [15]:
print(len(df))

2405317


In [16]:
print(len(test2))

70665


### Etudes doublons 

In [17]:
df_final = pd.read_csv("testfinal.csv")

# Identifier les doublons (conservés malgré le drop dans le script précédent ?)
doublons = df_final[df_final.duplicated(
    subset=['Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere'], 
    keep=False
)]


/tmp/ipykernel_179738/2613097225.py:1: DtypeWarning: Columns (15,58,60,62,65,67,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv("testfinal.csv")


In [21]:
from IPython.display import display

display(doublons.sort_values(by='Adresse_Normalisee').head(100))


,Date_réception_DPE,Date_établissement_DPE,Modèle_DPE,Date_fin_validité_DPE,Version_DPE,Méthode_application_DPE,Etiquette_DPE,Etiquette_GES,Année_construction,Type_bâtiment,...,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,Adresse
59972,2022-02-15,2022-02-15,DPE 3CL 2021 méthode logement,2032-02-14,2.0,dpe appartement individuel,E,B,2009.0,appartement,...,3,4.0,Local industriel. commercial ou assimilé,NaN,20.0,0.0,NaN,NaN,NaN,"1 ALL ANTOINE DE BOURBON, 64320 IDRON"
59979,2022-06-03,2022-06-03,DPE 3CL 2021 méthode logement,2032-06-02,2.1,dpe appartement individuel,E,B,2009.0,appartement,...,3,4.0,Local industriel. commercial ou assimilé,NaN,20.0,0.0,NaN,NaN,NaN,"1 ALL ANTOINE DE BOURBON, 64320 IDRON"
59980,2022-10-04,2022-10-04,DPE 3CL 2021 méthode logement,2032-10-03,2.2,dpe appartement individuel,E,B,2009.0,appartement,...,3,4.0,Local industriel. commercial ou assimilé,NaN,20.0,0.0,NaN,NaN,NaN,"1 ALL ANTOINE DE BOURBON, 64320 IDRON"
59974,2022-06-03,2022-06-03,DPE 3CL 2021 méthode logement,2032-06-02,2.1,dpe appartement individuel,D,B,2009.0,appartement,...,3,4.0,Local industriel. commercial ou assimilé,NaN,20.0,0.0,NaN,NaN,NaN,"1 ALL ANTOINE DE BOURBON, 64320 IDRON"
59970,2022-02-18,2022-02-18,DPE 3CL 2021 méthode logement,2032-02-17,2.0,dpe appartement individuel,D,B,2009.0,appartement,...,2,4.0,Local industriel. commercial ou assimilé,NaN,35.0,0.0,NaN,NaN,NaN,"1 ALL ANTOINE DE BOURBON, 64320 IDRON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2548,2022-01-13,2022-01-13,DPE 3CL 2021 méthode logement,2032-01-12,2.0,dpe appartement individuel,E,D,1930.0,appartement,...,1,2.0,Appartement,NaN,60.0,3.0,NaN,NaN,NaN,"1 AV DE LA REDOUTE, 92600 ASNIERES-SUR-SEINE"
2545,2022-01-13,2022-01-13,DPE 3CL 2021 méthode logement,2032-01-12,2.0,dpe appartement individuel,E,D,1930.0,appartement,...,1,2.0,Appartement,NaN,60.0,3.0,NaN,NaN,NaN,"1 AV DE LA REDOUTE, 92600 ASNIERES-SUR-SEINE"
31670,2022-01-28,2022-01-28,DPE 3CL 2021 méthode logement,2032-01-27,2.0,dpe appartement individuel,G,D,1950.0,appartement,...,2,2.0,Appartement,NaN,12.0,1.0,NaN,NaN,NaN,"1 AV DES CHARMETTES, 95400 VILLIERS-LE-BEL"
31671,2022-01-18,2022-01-18,DPE 3CL 2021 méthode logement,2032-01-17,2.0,dpe appartement individuel,G,D,1948.0,appartement,...,2,2.0,Appartement,NaN,12.0,1.0,NaN,NaN,NaN,"1 AV DES CHARMETTES, 95400 VILLIERS-LE-BEL"


In [90]:
result_df = test2.drop_duplicates(subset=['Date_réception_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere', 'N°_étage_appartement', 'Etiquette_DPE', "Complément_d'adresse_logement", "Complément_d'adresse_bâtiment", "Date_établissement_DPE"])

In [91]:
result_df.to_csv("result5%.csv", index=False)

In [92]:
print(len(result_df))

77148


In [47]:
df_final = pd.read_csv("testfinal.csv")

# Identifier les doublons (conservés malgré le drop dans le script précédent ?)
doublons1 = df_final[df_final.duplicated(
    subset=['Date_réception_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere', 'N°_étage_appartement', 'Etiquette_DPE', "Complément_d'adresse_logement"], 
    keep=False
)]

/tmp/ipykernel_179738/3911531778.py:1: DtypeWarning: Columns (15,58,60,62,65,67,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv("testfinal.csv")


In [ ]:
from IPython.display import display
pd.set_option('display.max_columns', None)

display(doublons1.sort_values(by='Adresse_Normalisee').head(100))

,Date_réception_DPE,Date_établissement_DPE,Modèle_DPE,Date_fin_validité_DPE,Version_DPE,Méthode_application_DPE,Etiquette_DPE,Etiquette_GES,Année_construction,Type_bâtiment,Période_construction,Surface_habitable_logement,Adresse_brute,Nom__commune_(BAN),Code_INSEE_(BAN),N°_voie_(BAN),Identifiant__BAN,Adresse_(BAN),Code_postal_(BAN),Score_BAN,Nom__rue_(BAN),Coordonnée_cartographique_X_(BAN),Coordonnée_cartographique_Y_(BAN),Code_postal_(brut),N°_étage_appartement,Nom_résidence,Cage_d'escalier,Complément_d'adresse_logement,Statut_géocodage,Nom__commune_(Brut),N°_département_(BAN),N°_région_(BAN),Complément_d'adresse_bâtiment,Adresse_Normalisee,Identifiant de document,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,Commune,Code departement,Code commune,Prefixe de section,Section,No plan,No Volume,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,3eme lot,Surface Carrez du 3eme lot,4eme lot,Surface Carrez du 4eme lot,5eme lot,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,Adresse
59384,2022-10-18,2022-10-18,DPE 3CL 2021 méthode logement,2032-10-17,2.2,dpe appartement individuel,D,B,2009.0,appartement,2006-2012,19.6,Antoine de Bourbon,Idron,64269,1,64269_0157_00001,1 allée Antoine de Bourbon 64320 Idron,64320.0,0.63,allée Antoine de Bourbon,431313.41,6250467.53,64320,1.0,Confort Suite,NaN,131,adresse non géocodée ban car aucune correspond...,IDRON,64,75.0,NaN,1 allee antoine de bourbon 64320 idron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,22/03/2022,Vente,"43060,00",1.0,NaN,ALL,0157,ANTOINE DE BOURBON,64320,IDRON,64,269,NaN,BO,141,NaN,164.0,19.46,230.0,NaN,231.0,NaN,NaN,NaN,NaN,NaN,3,4.0,Local industriel. commercial ou assimilé,NaN,20.0,0.0,NaN,NaN,NaN,"1 ALL ANTOINE DE BOURBON, 64320 IDRON"
59371,2022-10-18,2022-10-18,DPE 3CL 2021 méthode logement,2032-10-17,2.2,dpe appartement individuel,D,B,2009.0,appartement,2006-2012,19.6,Antoine de Bourbon,Idron,64269,1,64269_0157_00001,1 allée Antoine de Bourbon 64320 Idron,64320.0,0.63,allée Antoine de Bourbon,431313.41,6250467.53,64320,1.0,Confort Suite,NaN,131,adresse non géocodée ban car aucune correspond...,IDRON,64,75.0,NaN,1 allee antoine de bourbon 64320 idron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,22/03/2022,Vente,"43060,00",1.0,NaN,ALL,0157,ANTOINE DE BOURBON,64320,IDRON,64,269,NaN,BO,141,NaN,164.0,19.46,230.0,NaN,231.0,NaN,NaN,NaN,NaN,NaN,3,4.0,Local industriel. commercial ou assimilé,NaN,20.0,0.0,NaN,NaN,NaN,"1 ALL ANTOINE DE BOURBON, 64320 IDRON"
47213,2022-04-12,2022-04-12,DPE 3CL 2021 méthode logement,2032-04-11,2.1,dpe appartement individuel,C,C,2005.0,appartement,2001-2005,27.4,1 Allée Antoine Osete,Toulouse,31555,1,31555_0397_00001,1 Allée Antoine Osete 31100 Toulouse,31100.0,0.49,Allée Antoine Osete,571017.15,6278278.23,31100,NaN,Résidence Hotelière Hippodrome,NaN,NaN,adresse non géocodée ban car aucune correspond...,TOULOUSE,31,76.0,NaN,1 allee antoine osete 31100 toulouse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,19/07/2022,Vente,"89170,00",1.0,NaN,ALL,0397,ANTOINE OSETE,31100,TOULOUSE,31,555,844.0,AR,67,NaN,198.0,27.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,Appartement,NaN,27.0,2.0,NaN,NaN,NaN,"1 ALL ANTOINE OSETE, 31100 TOULOUSE"
47215,2022-04-12,2022-04-12,DPE 3CL 2021 méthode logement,2032-04-11,2.1,dpe appartement individuel,C,C,2005.0,appartement,2001-2005,27.4,1 Allée Antoine Osete,Toulouse,31555,1,31555_0397_00001,1 Allée Antoine Osete 31100 Toulouse,31100.0,0.49,Allée Antoine Osete,571017.15,6278278.23,31100,NaN,Résidence Hotelière Hippodrome,NaN,NaN,adresse non géocodée ban car aucune correspond...,TOULOUSE,31,76.0,NaN,1 allee antoine osete 31100 toulouse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,19/07/2022,Vente,"89170,00",1.0,NaN,ALL,0397,ANTOINE OSETE,31100,TOULOUSE,31,555,844.0,AR,67,Na

## TEST AVEC 10% POUR LES SURFACES

In [94]:
def test_match3(vf, df):
    merged = []


    #boucle par département
    for department in df['N°_département_(BAN)'].unique():  
        print(f"Traitement du département : {department}")
        
        #filtre
        vf_dept = vf[vf['Code postal'].astype(str).str.startswith(department)].copy()
        df_dept = df[df['N°_département_(BAN)']==department].copy()

        #normalisation adresses
        vf_dept['Adresse'] = vf_dept['Adresse'].str.strip().str.replace(r'\s+', ' ', regex=True)
        vf_dept['Adresse_Normalisee'] = vf_dept.apply(lambda row: normalize_vf_address(row['Adresse'], row['Code postal']), axis=1)
        df_dept['Adresse_Normalisee'] = df_dept['Adresse_(BAN)'].apply(normalize_address)

        adresse_counts = vf_dept['Adresse_Normalisee'].dropna().value_counts()
        unique1 = list(adresse_counts[adresse_counts == 1].index)
        doublons1 = list(adresse_counts[adresse_counts > 1].index)
        final = unique1 + doublons1
        set_final = set(final)

        adresse_counts2 = df_dept['Adresse_Normalisee'].dropna().value_counts()
        unique2 = list(adresse_counts2[adresse_counts2 == 1].index)
        doublons2 = list(adresse_counts2[adresse_counts2 > 1].index)
        final2 = unique2 + doublons2
        set_final2 = set(final2)

        commun = set_final.intersection(set_final2)

        vf_dept['Surface Carrez du 1er lot'] = pd.to_numeric(
            vf_dept['Surface Carrez du 1er lot'].astype(str).str.replace(',', '.'), errors='coerce'
        )
        df_dept['Surface_habitable_logement'] = pd.to_numeric(
            df_dept['Surface_habitable_logement'].astype(str).str.replace(',', '.'), errors='coerce'
        )

        for adresse in commun:
            dfsub = df_dept[df_dept['Adresse_Normalisee'] == adresse]
            vfsub = vf_dept[vf_dept['Adresse_Normalisee'] == adresse]

            for _, row2 in dfsub.iterrows():
                best_match = None
                best_value = -1

                for _, row1 in vfsub.iterrows():
                    surface1 = row1['Surface Carrez du 1er lot']
                    surface2 = row2['Surface_habitable_logement']

                    #si surfaces identiques on match direct
                    if surface1 == surface2:
                        best_match = row1
                        break

                    #ecart inf à 10%
                    if abs(surface1 - surface2) / max(surface1, surface2) < 0.1:
                        valeur_fonciere = pd.to_numeric(str(row1.get('Valeur fonciere', 0)).replace(',', '.'), errors='coerce')
                        if valeur_fonciere > best_value:
                            best_value = valeur_fonciere
                            best_match = row1

                if best_match is not None:
                    merged.append({**row2.to_dict(), **best_match.to_dict()})

    #df des résultats fusionnés
    df = pd.DataFrame(merged)
    return df

#application fonction
test3 = test_match3(vf, df)

Traitement du département : 13
Traitement du département : 92
Traitement du département : 69
Traitement du département : 03
Traitement du département : 56
Traitement du département : 75
Traitement du département : 29
Traitement du département : 91
Traitement du département : 62
Traitement du département : 21
Traitement du département : 11
Traitement du département : 02
Traitement du département : 59
Traitement du département : 94
Traitement du département : 44
Traitement du département : 50
Traitement du département : 37
Traitement du département : 81
Traitement du département : 67
Traitement du département : 93
Traitement du département : 83
Traitement du département : 63
Traitement du département : 95
Traitement du département : 76
Traitement du département : 33
Traitement du département : 78
Traitement du département : 38
Traitement du département : 51
Traitement du département : 57
Traitement du département : 77
Traitement du département : 35
Traitement du département : 54
Traiteme

In [95]:
print(len(test3))

94314


In [96]:
result10 = test3.drop_duplicates(subset=['Date_réception_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere', 'N°_étage_appartement', 'Etiquette_DPE', "Complément_d'adresse_logement", "Complément_d'adresse_bâtiment", "Date_établissement_DPE"])

In [97]:
print(len(result10))

88831


In [98]:
result10.to_csv("result10%.csv", index=False)

## TEST SEUIL DE 3%

In [27]:
def test_match4(vf, df):
    merged = []


    #boucle par département
    for department in df['N°_département_(BAN)'].unique():  
        print(f"Traitement du département : {department}")
        
        #filtre
        vf_dept = vf[vf['Code postal'].astype(str).str.startswith(department)].copy()
        df_dept = df[df['N°_département_(BAN)']==department].copy()

        #normalisation adresses
        vf_dept['Adresse'] = vf_dept['Adresse'].str.strip().str.replace(r'\s+', ' ', regex=True)
        vf_dept['Adresse_Normalisee'] = vf_dept.apply(lambda row: normalize_vf_address(row['Adresse'], row['Code postal']), axis=1)
        df_dept['Adresse_Normalisee'] = df_dept['Adresse_(BAN)'].apply(normalize_address)

        adresse_counts = vf_dept['Adresse_Normalisee'].dropna().value_counts()
        unique1 = list(adresse_counts[adresse_counts == 1].index)
        doublons1 = list(adresse_counts[adresse_counts > 1].index)
        final = unique1 + doublons1
        set_final = set(final)

        adresse_counts2 = df_dept['Adresse_Normalisee'].dropna().value_counts()
        unique2 = list(adresse_counts2[adresse_counts2 == 1].index)
        doublons2 = list(adresse_counts2[adresse_counts2 > 1].index)
        final2 = unique2 + doublons2
        set_final2 = set(final2)

        commun = set_final.intersection(set_final2)

        vf_dept['Surface Carrez du 1er lot'] = pd.to_numeric(
            vf_dept['Surface Carrez du 1er lot'].astype(str).str.replace(',', '.'), errors='coerce'
        )
        df_dept['Surface_habitable_logement'] = pd.to_numeric(
            df_dept['Surface_habitable_logement'].astype(str).str.replace(',', '.'), errors='coerce'
        )

        for adresse in commun:
            dfsub = df_dept[df_dept['Adresse_Normalisee'] == adresse]
            vfsub = vf_dept[vf_dept['Adresse_Normalisee'] == adresse]

            for _, row2 in dfsub.iterrows():
                best_match = None
                best_value = -1

                for _, row1 in vfsub.iterrows():
                    surface1 = row1['Surface Carrez du 1er lot']
                    surface2 = row2['Surface_habitable_logement']

                    #si surfaces identiques on match direct
                    if surface1 == surface2:
                        best_match = row1
                        break

                    #ecart inf à 3%
                    if abs(surface1 - surface2) / max(surface1, surface2) < 0.03:
                        valeur_fonciere = pd.to_numeric(str(row1.get('Valeur fonciere', 0)).replace(',', '.'), errors='coerce')
                        if valeur_fonciere > best_value:
                            best_value = valeur_fonciere
                            best_match = row1

                if best_match is not None:
                    merged.append({**row2.to_dict(), **best_match.to_dict()})

    #df des résultats fusionnés
    df = pd.DataFrame(merged)
    return df

#application fonction
test4 = test_match4(vf, df)

Traitement du département : 13
Traitement du département : 92
Traitement du département : 69
Traitement du département : 03
Traitement du département : 56
Traitement du département : 75
Traitement du département : 29
Traitement du département : 91
Traitement du département : 62
Traitement du département : 21
Traitement du département : 11
Traitement du département : 02
Traitement du département : 59
Traitement du département : 94
Traitement du département : 44
Traitement du département : 50
Traitement du département : 37
Traitement du département : 81
Traitement du département : 67
Traitement du département : 93
Traitement du département : 83
Traitement du département : 63
Traitement du département : 95
Traitement du département : 76
Traitement du département : 33
Traitement du département : 78
Traitement du département : 38
Traitement du département : 51
Traitement du département : 57
Traitement du département : 77
Traitement du département : 35
Traitement du département : 54
Traiteme

In [28]:
print(len(test4))

73502


In [29]:
result3 = test4.drop_duplicates(subset=['Date_réception_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere', 'N°_étage_appartement', 'Etiquette_DPE', "Complément_d'adresse_logement", "Complément_d'adresse_bâtiment", "Date_établissement_DPE"])

In [30]:
print(len(result3))

69313


In [31]:
result3.to_csv("result3%.csv", index=False)

## TEST AUTRES ANNEES

Il semble que nous ne disposions que des données à partir de 2021, nous pouvons tout de même vérifier que c'est bien le cas car les fichiers valeurs foncières s'étendent jusqu'à 2014.

In [ ]:
df['Date_réception_DPE'] = pd.to_datetime(df['Date_réception_DPE'], errors='coerce')
df2014 = df[df['Date_réception_DPE'].dt.year == 2014].copy()

In [35]:
print(len(df2014))

0


In [ ]:
df2015 = df[df['Date_réception_DPE'].dt.year == 2015].copy()

In [37]:
print(len(df2015))

0


In [ ]:
df2016 = df[df['Date_réception_DPE'].dt.year == 2016].copy()

In [39]:
print(len(df2016))

0


In [ ]:
df2017 = df[df['Date_réception_DPE'].dt.year == 2017].copy()

In [41]:
print(len(df2017))

0


In [ ]:
df2018 = df[df['Date_réception_DPE'].dt.year == 2018].copy()

In [43]:
print(len(df2018))

0


In [ ]:
df2019 = df[df['Date_réception_DPE'].dt.year == 2019].copy()

In [45]:
print(len(df2019))

0


In [ ]:
df2020 = df[df['Date_réception_DPE'].dt.year == 2020].copy()

In [47]:
print(len(df2020))

0


In [ ]:
df2021 = df[df['Date_réception_DPE'].dt.year == 2021].copy()

In [49]:
print(len(df2021))

677703


In [ ]:
df2023= df[df['Date_réception_DPE'].dt.year == 2023].copy()

In [51]:
print(len(df2023))

1438813


Nous allons donc pouvoir seulement compiler les données pour les années : 2021, 2022,2023.

### ANALYSE DES DONNEES COMMUNES ENTRE DF ET DF2 (LOGEMENTS EXISTANTS ET NEUFS)